# Implementation of the EBfilter by genomon

* EBrun (originally EBFilter) is an argparse wrapper passing command line arguments to run.py (is not needed for internal use)
* passed arguments:
    * targetMutationFile: the .vcf or .anno containing the mutations – needed --> mut_file
    * targetBamPath: path to the tumor bam file (+.bai) – needed --> tumor_bam
    * controlBamPathList: text list of path to PoN bam files (+ .bai) – needed --> pon_list
    * outputPath: clear  – needed --> output_path
    * -f option for anno or vcf – not needed --> will be inferred from .ext
    * thread_num: –not needed --> taken from config
    * -q option for quality threshold – not needed --> default _q config
    * -Q option for base quality threshold - not needed --> default _Q from config
    * --ff option for filter flags – not needed because of preprocessing??
    * --loption for samtools mpileup -l option – must elaborate..
    * --region option for restriction of regions on mpileup -l – must elaborate..
    * --debug – not needed

## Initiation

### imports

In [1]:
import os
from code import run

### snakemake config

In [2]:
config = {'EB':{'run': True}}
params = {}
params['map_quality'] = 20
params['base_quality'] = 15
params['filter_flags'] = 'UNMAP,SECONDARY,QCFAIL,DUP'
params['loption'] = True
config['EB']['threads'] = 1
config['EB']['params'] = params
config['annovar'] = {'sep': '\t'}

### load the config and GLOBAL STATE into variable _

In [3]:
debug_mode = True
args = {}
params = config['EB']['params']
threads = config['EB']['threads']
sep = config['annovar']['sep']
_q = str(params['map_quality'])  # mapping quality=20
_Q = params['base_quality']      # base quality=15
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = params['filter_flags']     # 'UNMAP,SECONDARY,QCFAIL,DUP'
is_loption = params['loption']   # True
log_path = 'output/logs'         # log_file: 'output/logs/'



### function args
will be passed during the function call

In [4]:
args['mut_file'] = '/Users/mahtin/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv'
args['tumor_bam'] = '/Users/mahtin/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/bam/test_Rel1.bam'
args['pon_list'] = '/Users/mahtin/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['output_path'] = 'output/test_rel_eb.csv'
args['region'] = ''
log_file = os.path.join(log_path, f"{os.path.splitext(os.path.basename(args['output_path']))[0]}.log")
# dump into _
_ = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'ff':_ff, 'log':log_file, 'threads':threads, 'sep': sep, 'debug_mode':debug_mode}

## Single threading

In [11]:
##### RUNNIT
_['threads'] = 1
_['debug_mode'] = False
run.main(args,_)
!ls output

AML033_rel_eb.csv                logs
AML033_rel_eb.csv.control.pileup test_rel_eb.csv
AML033_rel_eb.csv.target.pileup


## Multithreading

In [9]:
_['threads'] = 3
_['debug_mode'] = False
run.main(args,_)
!ls output/

AML033_rel_eb.csv                logs
AML033_rel_eb.csv.control.pileup test_rel_eb.csv
AML033_rel_eb.csv.target.pileup


## using advanced dataframes for getting the pileup files 

### get anno and pileup as dataframes (too big?)

In [ ]:
anno = pd.read_csv('testdata/input.anno', sep='\t', header=None, names=['Chr','Start', 'End', 'ref', 'var'], dtype={'Chr': str, 'Start':int, 'End': int})
tumorpileup = pd.read_csv('output/tumor.pileup', sep='\t', header=None, names=['Chr', 'Start', 'ref', 'depth', 'reads', 'mapQ'], dtype={'Start':int, 'reads':str, 'mapQ': str})
pd.merge(anno,tumorpileup, on=['Chr', 'Start'])


In [ ]:
threads = 4
split = round(len(tumorpileup.index) / threads)
splits = np.array_split(tumorpileup, 4)

In [ ]:
import pandas as pd
anno = pd.read_csv('testdata/input.anno', sep='\t', header=None)
anno

In [ ]:
anno[1] = anno[1] - 1 - (anno[3] == '-')
anno[2] = anno[2] - (anno[3] == '-')
anno

In [ ]:
anno